### `Participation Rate` vs `Conditional Participation Rate`.

to help understand the different summing of rates

In [25]:
#import pathlib
import json
import nzshm_model as nm
import solvis
from solvis.fault_system_solution_helper import FaultSystemSolutionHelper


In [26]:
# OPTIONS
SINGLE_SOLUTION = False
SECTION_RATE = False # otherwise Parent rates

if SINGLE_SOLUTION:
    solution = solvis.InversionSolution.from_archive("NZSHM22_ScaledInversionSolution-QXV0b21hdGlvblRhc2s6MTEzMTM0.zip")
else:
    current_model = nm.get_model_version("NSHM_v1.0.4")
    slt = current_model.source_logic_tree
    csol = solvis.CompositeSolution.from_archive("NSHM_v1.0.4_CompositeSolution.zip", slt)
    solution = csol._solutions['CRU']

helper = FaultSystemSolutionHelper(solution)  
rate_column = "Annual Rate" if isinstance(solution, solvis.InversionSolution) else "rate_weighted_mean"


In [27]:
# get ruptures passing through a named parent fault
TARGET_FAULTS = ['Opotiki 3', 'Calypso 2']
TARGET_FAULTS = ['Opotiki 3'] 

ruptures = helper.ruptures_for_parent_fault_names(TARGET_FAULTS) # 

# # get rupture fault sections (rs) with rates for those ruptures
# df0 = solution.rs_with_rupture_rates
# rupture_sections_df = df0[df0["Rupture Index"].isin(ruptures)]

### calculate participation rate series

In [28]:
# for comparison
opo_section = 1426 # Opotiki 3, Subsection 2
cal_section = 217  # Calypso 2, Subsection 1
df0 = solution.rs_with_rupture_rates

print("summing the subsection rate")
print("===========================")

print("new sum() filtering by section ID: `Participation Rate` (all ruptures in the system)")
print(f'Opotiki rate: {df0[df0.section==opo_section][rate_column].sum()}')
print(f'Calypso rate: {df0[df0.section==cal_section][rate_column].sum()}')
print()
print("API sum() filtering by rupture ID: `Conditional Participation Rate` (filtered by rupture id or attribute)")
df0 = df0[df0["Rupture Index"].isin(ruptures)]
print(f'Opotiki rate: {df0[df0.section==opo_section][rate_column].sum()}')
print(f'Calypso rate: {df0[df0.section==cal_section][rate_column].sum()}')


summing the subsection rate
new sum() filtering by section ID: `Participation Rate` (all ruptures in the system)
Opotiki rate: 0.00017244191258214414
Calypso rate: 0.00041707666241563857

API sum() filtering by rupture ID: `Conditional Participation Rate` (filtered by rupture id or attribute)
Opotiki rate: 0.00017244191258214414
Calypso rate: 1.67943744600052e-05


In [29]:
## same for parent fault summation

opo_fault = 330 # Opotiki 3
cal_fault = 52  # Calypso 2, Subsection 1

df0 = solution.rs_with_rupture_rates
df1 = df0.join(solution.fault_sections[['ParentID']], on='section')
# df1[df1.ParentID==cal_fault]
# assert 0

df2 = df1[["Rupture Index", "ParentID", "section", rate_column]]
# parent_rates = fault_rates.groupby("ParentID").agg('sum')[rate_column]

print("summing the parent fault rate")
print("========================--===")

print("new sum() filtering by section ID: `Participation Rate` (all ruptures in the system)")
print(f'Opotiki rate: {df2[df2.ParentID==opo_fault][rate_column].sum()}')
print(f'Calypso rate: {df2[df2.ParentID==cal_fault][rate_column].sum()}')
print()
print("API sum() filtering by rupture ID: `Conditional Participation Rate` (filtered by rupture id or attribute)")
df3 = df2[df2["Rupture Index"].isin(ruptures)]
print(f'Opotiki rate: {df3[df3.ParentID==opo_fault][rate_column].sum()}')
print(f'Calypso rate: {df3[df3.ParentID==cal_fault][rate_column].sum()}')


summing the parent fault rate
========================--===
new sum() filtering by section ID: `Participation Rate` (all ruptures in the system)
Opotiki rate: 0.0005705734947696328
Calypso rate: 0.0008173590758815408

API sum() filtering by rupture ID: `Conditional Participation Rate` (filtered by rupture id or attribute)
Opotiki rate: 0.0005705734947696328
Calypso rate: 1.67943744600052e-05
